In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, log_loss, accuracy_score
from sklearn.preprocessing import  OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from tqdm import tqdm

In [2]:
hr = pd.read_csv("C:/Python/Cases/human-resources-analytics/HR_comma_sep.csv")
X = hr.drop('left', axis=1)
y = hr['left']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25,
                                                    stratify=y)

In [3]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
trns = make_column_transformer((ohe, make_column_selector(dtype_include=object) ), 
                               remainder='passthrough', verbose_feature_names_out=False)
trns = trns.set_output(transform='pandas')
X_train_trns = trns.fit_transform(X_train)
X_test_trns = trns.transform(X_test)

In [5]:
dtc = DecisionTreeClassifier(random_state=25)
knn = KNeighborsClassifier()
nb = GaussianNB()
bagg = BaggingClassifier(estimator=nb, n_estimators=10, random_state=25)
bagg.fit(X_train_trns, y_train)
y_pred = bagg.predict(X_test_trns)
print( classification_report(y_test, y_pred) )

              precision    recall  f1-score   support

           0       0.90      0.70      0.79      3429
           1       0.44      0.76      0.56      1070

    accuracy                           0.71      4499
   macro avg       0.67      0.73      0.67      4499
weighted avg       0.79      0.71      0.73      4499



In [6]:
lr = LogisticRegression()

In [8]:
est_list = [nb, dtc, knn, lr]
n_est = [10, 15, 25, 50]
scores = []
for e in tqdm(est_list):
    for n in n_est:
        bagg = BaggingClassifier(random_state=25, n_estimators=n, estimator=e )
        bagg.fit(X_train_trns, y_train)
        y_pred_prob = bagg.predict_proba(X_test_trns)
        scores.append([e, n, log_loss(y_test, y_pred_prob)])
df_scores = pd.DataFrame( scores, columns=['Estimator','B-Samples','score'] )
df_scores.sort_values('score')

 75%|███████▌  | 3/4 [00:11<00:04,  4.80s/it]c:\Python\anaconda3\envs\hf\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Python\anaconda3\envs\hf\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.o

,Estimator,B-Samples,score
7,DecisionTreeClassifier(random_state=25),50,0.128360
6,DecisionTreeClassifier(random_state=25),25,0.149958
5,DecisionTreeClassifier(random_state=25),15,0.158270
4,DecisionTreeClassifier(random_state=25),10,0.173862
11,KNeighborsClassifier(),50,0.417508
15,LogisticRegression(),50,0.429781
14,LogisticRegression(),25,0.430028
13,LogisticRegression(),15,0.430121
12,LogisticRegression(),10,0.430184
10,KNeighborsClassifier(),25,0.452318


For the same estimator, trying different parameter values

In [9]:
depths = [None, 3, 5, 7]
scores = []
for d in tqdm( depths ):
    dtc = DecisionTreeClassifier(random_state=25, max_depth=d)
    bagg = BaggingClassifier(random_state=25, n_estimators=50, estimator=dtc )
    bagg.fit(X_train_trns, y_train)
    y_pred_prob = bagg.predict_proba(X_test_trns)
    scores.append([d, log_loss(y_test, y_pred_prob)])
df_scores = pd.DataFrame( scores, columns=['depth', 'score'] )
df_scores.sort_values('score')

100%|██████████| 4/4 [00:02<00:00,  1.49it/s]


,depth,score
3,7.0,0.079241
2,5.0,0.088876
0,NaN,0.128360
1,3.0,0.138152
